In [143]:
import numpy as np
from scipy import stats

In [144]:
survival = np.array([49, 58, 75, 110, 112, 132, 151, 276, 281, 362])

In [145]:
m0 = 200
p_value = stats.wilcoxon(survival - m0).pvalue
print(f"p-value: {p_value:.4f}")

p-value: 0.2845


In [146]:
cutting = np.array([17, 18, 18, 15, 12, 4, 14, 15, 10])
non_cutting = np.array([22, 22, 15, 13, 19, 19, 18, 20, 21, 13, 13, 15])

In [147]:
p_value = stats.mannwhitneyu(cutting, non_cutting, alternative='less').pvalue
print(f"p-value: {p_value:.4f}")

p-value: 0.0290


In [148]:
import pandas as pd

In [149]:
data = pd.read_csv('challenger.txt', delimiter='\t')
data.head()

,Unnamed: 0,Temperature,Incident
0,Apr12.81,18.9,0
1,Nov12.81,21.1,1
2,Mar22.82,20.6,0
3,Nov11.82,20.0,0
4,Apr04.83,19.4,0


In [150]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [151]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [152]:
data_incident = data[data.Incident == 0].Temperature.values
data_success = data[data.Incident == 1].Temperature.values

In [153]:
np.random.seed(0)

incident_mean_scores = list(map(np.mean, get_bootstrap_samples(data_incident, 1000)))
success_mean_scores = list(map(np.mean, get_bootstrap_samples(data_success, 1000)))

delta_mean_scores = list(map(lambda x: x[1] - x[0], zip(success_mean_scores, incident_mean_scores)))
interval = stat_intervals(delta_mean_scores, 0.05)

print(f"Stat interval: ({interval[0]:.4f}, {interval[1]:.4f})")

Stat interval: (1.4230, 7.9386)


In [154]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

In [155]:
def get_random_combinations(n1, n2, max_combinations):
    index = list(range(n1 + n2))
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

In [156]:
def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

In [157]:
def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [164]:
np.random.seed(0)

p_value = permutation_test(data_success, data_incident, max_permutations = 10000)
print(f"p-value: {p_value:.4f}")

p-value: 0.0057
